In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def file_path(category, data_type):
  return f'/content/drive/MyDrive/ShoppingPulse/datasets/raw/interactions_{data_type}/flattened_data_{category}.parquet'

In [ ]:
def load_and_preprocess_data(category):

    dataset_path = file_path(category, 'train')
    df = pd.read_parquet(dataset_path)

    df['user_id'] = df['user_id'].astype(str)
    df['parent_asin'] = df['parent_asin'].astype(str)
    df['rating'] = df['rating'].astype(float)
    df['timestamp'] = df['timestamp'].apply(lambda x: pd.Timestamp(int(x), unit='ms'))

    sparse_matrix = df.pivot(index='user_id', columns='parent_asin', values='rating').fillna(0)
    R = sparse_matrix.values

    return sparse_matrix, R

def build_autoencoder(input_dim, encoding_dim):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)

    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)

    autoencoder.compile(optimizer=Adam(), loss='mean_squared_error')

    return autoencoder, encoder

def train_autoencoder(autoencoder, data, epochs=50, batch_size=256):
    autoencoder.fit(data, data, epochs=epochs, batch_size=batch_size, shuffle=True, validation_split=0.1)

def generate_predictions(autoencoder, data):
    predictions = autoencoder.predict(data)
    return predictions

def autoencoder_model(category, encoding_dim=50, epochs=50, batch_size=256):
    sparse_matrix, R = load_and_preprocess_data(category)

    input_dim = R.shape[1]
    autoencoder, encoder = build_autoencoder(input_dim, encoding_dim)

    train_autoencoder(autoencoder, R, epochs=epochs, batch_size=batch_size)

    predicted_ratings = generate_predictions(autoencoder, R)
    predicted_ratings_df = pd.DataFrame(predicted_ratings, columns=sparse_matrix.columns, index=sparse_matrix.index)

    return predicted_ratings_df

In [ ]:
def evaluate_recommendations(predicted_ratings_df, validation_df):
    users_in_both = set(predicted_ratings_df.index).intersection(set(validation_df['user_id']))

    recommendations = {}
    for user in users_in_both:
        sorted_items = predicted_ratings_df.loc[user].sort_values(ascending=False).index.tolist()
        recommendations[user] = sorted_items

    results = {}
    for user in users_in_both:
        recommended_items = set(recommendations[user])
        actual_purchases = set(validation_df[validation_df['user_id'] == user]['parent_asin'])
        intersection_count = len(recommended_items.intersection(actual_purchases))
        total_purchases = len(actual_purchases)
        results[user] = intersection_count / total_purchases if total_purchases > 0 else 0

    return results

In [ ]:
category = 'Gift_Cards'
predicted_ratings_df = autoencoder_model(category)
validation_df = pd.read_parquet(file_path(category, 'valid'))
evaluation_results = evaluate_recommendations(predicted_ratings_df, validation_df)
print(evaluation_results)

Epoch 1/50
387/387 [==============================] - 5s 12ms/step - loss: 0.0551 - val_loss: 0.0210
Epoch 2/50
387/387 [==============================] - 4s 10ms/step - loss: 0.0198 - val_loss: 0.0193
Epoch 3/50
387/387 [==============================] - 4s 10ms/step - loss: 0.0189 - val_loss: 0.0188
Epoch 4/50
387/387 [==============================] - 6s 15ms/step - loss: 0.0186 - val_loss: 0.0186
Epoch 5/50
387/387 [==============================] - 3s 8ms/step - loss: 0.0185 - val_loss: 0.0185
Epoch 6/50
387/387 [==============================] - 3s 8ms/step - loss: 0.0184 - val_loss: 0.0184
Epoch 7/50
387/387 [==============================] - 3s 9ms/step - loss: 0.0183 - val_loss: 0.0183
Epoch 8/50
387/387 [==============================] - 5s 12ms/step - loss: 0.0182 - val_loss: 0.0183
Epoch 9/50
387/387 [==============================] - 4s 11ms/step - loss: 0.0182 - val_loss: 0.0181
Epoch 10/50
387/387 [==============================] - 5s 12ms/step - loss: 0.0180 - val_loss: